#### 5.12.2021 - Hydrothermal Venture

#### Part 1

In [ ]:
from dataclasses import dataclass


@dataclass
class Point:
    x: int
    y: int


In [ ]:
@dataclass
class Vent:
    start: Point
    end: Point

    @property
    def diagnoal(self) -> bool:
        return not self.horizontal and not self.vertical

    @property
    def horizontal(self) -> bool:
        return self.start.y == self.end.y

    @property
    def vertical(self) -> bool:
        return self.start.x == self.end.x

    @property
    def max_x(self) -> int:
        return max(self.start.x, self.end.x)

    @property
    def max_y(self) -> int:
        return max(self.start.y, self.end.y)


In [ ]:
class VentSystem:
    def __init__(self):
        self.vents = []
        self.row_size = 0
        self.col_size = 0

    def append(self, vent: Vent):
        self.vents.append(vent)
        self.row_size = max(self.row_size, vent.max_x + 1)
        self.col_size = max(self.col_size, vent.max_y + 1)

    def append_str(self, vent_string: str):
        self.append(
            Vent(
                *map(lambda point_string:
                     Point(
                         *map(int, point_string.strip().split(','))
                     ),
                     vent_string.split('->'))
            )
        )

    def generate_map(self) -> list[list[int]]:
        world: list[list[int]] = [[0] * self.row_size
                                  for _ in range(self.col_size)]

        def bidirectional_range(a, b):
            return range(a, b + 1, 1) if a <= b else range(a, b - 1, -1)

        for vent in self.horizontal:
            for x in bidirectional_range(vent.start.x, vent.end.x):
                world[vent.start.y][x] += 1

        for vent in self.vertical:
            for y in bidirectional_range(vent.start.y, vent.end.y):
                world[y][vent.start.x] += 1

        for vent in self.diagonal:
            for x, y in zip(bidirectional_range(vent.start.x, vent.end.x),
                            bidirectional_range(vent.start.y, vent.end.y)):
                world[y][x] += 1
        return world

    @property
    def horizontal(self):
        return filter(lambda vent: vent.horizontal, self.vents)

    @property
    def vertical(self):
        return filter(lambda vent: vent.vertical, self.vents)

    @property
    def diagonal(self):
        return filter(lambda vent: vent.diagnoal, self.vents)


In [ ]:
with open('input_5/input.txt') as input_file:
    vents: VentSystem = VentSystem()

    data = input_file.readlines()
    for vent_string in data:
        vents.append_str(vent_string.strip())

    world: list[list[int]] = vents.generate_map()

    def flatten_list(lst: list[list]) -> list:
        return [item for sublist in lst for item in sublist]

    total = len([number for number in flatten_list(world) if number > 1])
    print("Dangerous areas: ", total)
